<a href="https://colab.research.google.com/github/freguti/progettoML/blob/main/ProgettoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

STEP:

1 - Dovremo prendere, per ogni super categoria, tutte le annotazioni e associarle alla super categoria corrispondente.

2 -  fare il fine-tuning del modello di CTRL tramite le annotazioni associate alla super categoria e generare del testo

    2.1 - modficare i control code durante la fase di fine tuning, in modo tale da inserire i nostri.  

3 - Valutare il testo generato tramite una versione modificata di TexyGen.

Dubbi: 
- bisogna usare le categorie o le super categorie? 

Commenti:

Al momento ho creato una demo che mostra le super categorie e prende le annotazioni di una categoria fissa.

CTRL sono riuscito solo a farlo girare in modalità low memory, ma è molto più inefficiente


**SIMONE:** guardare se la computazione della loss nella fase di fintuning sia corretta.

provare ad eseguire il finetuning con il trainer

la generazione genera parole a caso, non so se gli devo dare l'ultima parola generata o la frase generata fino a quel momento.


#Prepare environment

In [1]:
%%capture

from google.colab import drive
from pycocotools.coco import COCO
import os
import subprocess
!pip install transformers
from torch.utils.data import DataLoader
from transformers import AdamW, Trainer, TrainingArguments
import pandas as pd
import torch
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu,SmoothingFunction
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

NUM_LINES_PER_ITERATION = 15

In [ ]:
drive.mount('/content/drive', force_remount=True)

In [ ]:
#https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoDemo.ipynb demo coco API
def init_COCO():           
  dataDir='/content/drive/MyDrive/Machine Learning/Progetto'
  dataType='val2017'
  modelDir = dataDir + '/model/' 
  instFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
  annFile = '{}/annotations/captions_{}.json'.format(dataDir,dataType)
  
  coco=COCO(instFile) #modello con le foto e le categorie
  coco_ann=COCO(annFile) #modello con le annotazioni
  return coco,coco_ann

def choose_cat(cats):
  cat_choose = ""
  while cat_choose not in cats and cat_choose != "exit":
    cat_choose = input("Choose a category from this list {0}: \nPrint 'exit' to quit\n" .format(cats))
    if cat_choose not in cats and cat_choose != "exit":
      print("This category is not valid")
  
  print("category choosen: {0}".format(cat_choose))
  return cat_choose
  

# Prepare Dataset

In [ ]:
# display COCO categories and supercategories
def get_categories(coco):
  print('Getting categories...')
  cats = coco.loadCats(coco.getCatIds())
  nms=[cat['name'] for cat in cats]

  supernms = set([cat['supercategory'] for cat in cats])
  return supernms

In [ ]:
# load and display caption annotations by category
def get_annotation(coco,coco_ann,catNms = ['outdoor']):
  print('Getting annotations for {0}'.format(catNms))
  catIds = coco.getCatIds(catNms=catNms); #seleziono gli id delle categorie che voglio analizzare
  imgIds = coco.getImgIds(catIds=catIds); #ottengo gli id delle immagini delle categorie desiderate tramite gli id delle categorie
  annIds = coco_ann.getAnnIds(imgIds=imgIds); #ottengo gli id delle annotazioni relative alle immagini, tramite gli id delle immagini

  anns = coco_ann.loadAnns(annIds)
  return anns

# CTRL

## Huggingface

### Prepare model

In [ ]:
# --- PREPARE model, tokenizer and device

# Labels da togliere
class TextSet(torch.utils.data.Dataset): 
  def __init__(self, text, labels = ""):
    print("TextSet inizializing...")
    self.labels = labels
    self.text = text

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    label = self.labels[idx]
    text = self.text[idx]
    sample = {"Text": text, "Class": label}
    return sample

class TextSet_v3(torch.utils.data.Dataset): 
  def __init__(self, text):
    self.text = text

  def __len__(self):
    return len(self.text)

  def __getitem__(self, idx):
    text = self.text[idx]
    return text

def init(pretrained = False):
  print("Start init CTRLLMHeadModel")
  from transformers import CTRLLMHeadModel, CTRLConfig, CTRLTokenizer
  import torch
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  tokenizer = CTRLTokenizer.from_pretrained('ctrl')
  if not pretrained:
    model = CTRLLMHeadModel.from_pretrained('ctrl', config = CTRLConfig(n_layer = 9))
    model.save_pretrained("/content/drive/MyDrive/Machine Learning/Progetto/Models/vanilla10")
  else:
    #loaded_model = torch.jit.load("/content/drive/MyDrive/Machine Learning/Progetto/pretrained.pt")
    model = CTRLLMHeadModel.from_pretrained("/content/drive/MyDrive/Machine Learning/Progetto/Models/vanilla10")
  return model, tokenizer, device


### Finetune

In [ ]:
def prepare_dataset_v2(annotations, category, tokenizer):
  annotations_ = annotations[0:15000]
  annotations_train = []
  annotations_val = []
  for idx in iter(range(len(annotations_))):
    annotations_[idx] = category + " " + annotations_[idx]['caption']
    #annotations_[idx] = tokenizer.encode(annotations_[idx], add_special_tokens=False, return_tensors="pt")
    if idx % 15 == 0:
      annotations_val.append(tokenizer(annotations_[idx], padding="max_length", add_special_tokens=False, return_tensors="pt"))
    else:
      annotations_train.append(tokenizer(annotations_[idx], padding="max_length" ,add_special_tokens=False, return_tensors="pt"))
  return annotations_train,annotations_val

def finetune_v2(model,tokenizer,device,annotations,category = 'outdoor' ):#,dataset_path = "/content/drive/MyDrive/Machine Learning/Progetto/prova_1_val.txt"):
  print("start finetune")
  train_dataset = []
  model.to(device)
  len_annotation = len(annotations)
  print(len(annotations))
  len_annotation = 15000
  model.train()
  #Se non esiste il control code lo aggiungo
  encoded_category = tokenizer.encode(category, add_special_tokens=False)
  if not any(encoded_category[0] == x for x in tokenizer.control_codes.values()):
    tokenizer.control_codes[category] = encoded_category[0]
  #separo i due dataset
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  train_dataset,val_dataset = prepare_dataset_v2(annotations, category, tokenizer)

  training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Machine Learning/Progetto/results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=300,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/content/drive/MyDrive/Machine Learning/Progetto/logs',            # directory for storing logs
    logging_steps=10,
  )
  print(train_dataset)
  trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset             # evaluation dataset
  )

  trainer.train()

def format_dataset(annotations,start,category):
  dataset = []
  labels = []
  end = start + NUM_LINES_PER_ITERATION
  if len(annotations) < end:
    end = len(annotations)
  annotation = annotations[start:end ]
  text = category
  for ann in annotation:
    text += " " + ann['caption']
  dataset.append(text)
  labels.append(category)
  return dataset,end,labels

def finetune(model,tokenizer,device,annotations,category = 'outdoor' ):#,dataset_path = "/content/drive/MyDrive/Machine Learning/Progetto/prova_1_val.txt"):
  print("start finetune")
  train_dataset = []
  model.to(device)
  len_annotation = len(annotations)
  model.train()
  #Se non esiste il control code lo aggiungo
  encoded_category = tokenizer.encode(category, add_special_tokens=False)
  if not any(encoded_category[0] == x for x in tokenizer.control_codes.values()):
    tokenizer.control_codes[category] = encoded_category[0]
  for epoch in range(3):
    print("EPOCH")
    index = 0
    while index < len_annotation:
      dataset,index,labels = format_dataset(annotations = annotations, start = index, category = category)
      print("{0} / {1}".format(index,len_annotation))
      text_labels_df = pd.DataFrame({'Text': dataset, 'Labels': labels})
      TD = TextSet(text_labels_df['Text'], text_labels_df['Labels'])
                            
      train_loader = DataLoader(TD, batch_size=1, shuffle=True)
      optim = AdamW(model.parameters(), lr=5e-5)
      for batch in train_loader: #da vedere: batch size a 1 iterando su ogni elemento del dataset; vedere se va bene montare piu frasi in un solo elemento 
        print(batch)
        batch_t = []
        optim.zero_grad()
        for b in batch["Text"]:
          batch_t = tokenizer(b, return_tensors="pt", truncation=True)
          print(type(batch_t))
        batch_t.to(device)
        outputs = model(**batch_t, labels=batch_t["input_ids"])
        loss = outputs[0]
        print(loss)
        loss.backward()
        optim.step()  

def format_dataset_v3(annotations,category,tokenizer):
  dataset = []
  for ann in annotations:
    text = category + " " + ann['caption']
    tokenizer_kwargs = {}
    dataset.append(text)
  return dataset

def finetune_v3(model, tokenizer, device, annotations, category = 'outdoor' ):#,dataset_path = "/content/drive/MyDrive/Machine Learning/Progetto/prova_1_val.txt"):
  print("start finetune")
  train_dataset = []
  model.to(device)
  len_annotation = len(annotations)
  model.train()
  #Se non esiste il control code lo aggiungo
  encoded_category = tokenizer.encode(category, add_special_tokens=False)
  if not any(encoded_category[0] == x for x in tokenizer.control_codes.values()):
    tokenizer.control_codes[category] = encoded_category[0]

  #tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  dataset = format_dataset_v3(annotations = annotations, category = category, tokenizer = tokenizer)
  text_labels_df = pd.DataFrame({'Text': dataset})
  TD = TextSet_v3(text_labels_df['Text'])                        
  train_loader = DataLoader(TD, batch_size=1, shuffle=True)
  optim = AdamW(model.parameters(), lr=1e-4)
  

  for epoch in range(3):
    n_steps = 0
    avg = 0
    print("EPOCH")
    for batch in train_loader: #da vedere: batch size a 1 iterando su ogni elemento del dataset; vedere se va bene montare piu frasi in un solo elemento 
      optim.zero_grad()
      batch_t = tokenizer(batch, return_tensors="pt", truncation=True)
      batch_t.to(device)
    
      outputs = model(**batch_t, labels=batch_t['input_ids'])
      loss = outputs[0]
      del batch_t
      del outputs
      torch.cuda.empty_cache()
      n_steps = n_steps + 1
      avg = avg + loss
      if n_steps >= 300:
        avg = avg/n_steps
        print(avg)
        n_steps = 0
        avg = 0
      loss.backward(retain_graph=False)
      optim.step() 
    avg = avg/n_steps
    print(avg) 

### Generation

In [ ]:
'''
def generate(model,tokenizer,device,first_word = "outdoor "):
  model.eval()
  word = ""
  phrase = ""
  for k in range(10):
    phrase = phrase + word + " "
    print("phrase: " + phrase)
    inputs = tokenizer.encode_plus(first_word + word, return_tensors="pt")
    inputs = inputs.to(device)
    outputs = model(**inputs,labels=inputs["input_ids"])
    word =  decode_word(output = outputs)
    print("word:" + word[0])
  return outputs

def decode_word(output):
  x = ""
  loss = output.loss
  logits = output.logits
  print(logits)
  softmax = F.softmax(logits, dim = -1)
  index = torch.argmax(softmax,dim = -1)
  x = tokenizer.decode(index[0])
  return x
'''

def prepare_ctrl_input(tokenizer, prompt_text):
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False)
  if not any(encoded_prompt[0] == x for x in tokenizer.control_codes.values()):
      print(tokenizer.control_codes.values())
  return prompt_text

def generate_v2(model,tokenizer,device,control_code = "outdoor"):
  #controllo che il control code esista
  model.to(device)
  preprocessed_prompt_text = prepare_ctrl_input(tokenizer, control_code)
  tokenizer_kwargs = {}
  encoded_prompt = tokenizer.encode(
      preprocessed_prompt_text, add_special_tokens=False, return_tensors="pt", **tokenizer_kwargs
  )

  if encoded_prompt.size()[-1] == 0:
    input_ids = None
  else:
    input_ids = encoded_prompt
  output_sequences = model.generate(
        input_ids=input_ids.to(device),
        max_length= 50 + len(encoded_prompt[0]),
        temperature=0.4,
        top_k=1,
        top_p=0.8,
        repetition_penalty=5.0,
        do_sample=True,
        num_return_sequences=1,
    )
  print(output_sequences)

  if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
      print(f"=== GENERATED SEQUENCE {generated_sequence_idx + 1} ===")
      generated_sequence = generated_sequence.tolist()

      # Decode text
      text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

      # Remove all text after the stop token
      text = text[: None]

      # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
      total_sequence = (
          control_code + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
      )

      generated_sequences.append(total_sequence)
      print(total_sequence)
  return total_sequence

### No texygen evaluation

In [ ]:
# BLEU or not BLEU ------>  https://towardsdatascience.com/evaluating-text-output-in-nlp-bleu-at-your-own-risk-e8609665a213
#UTILE PER COMPRENDERE BLEU N-GRAM https://stackoverflow.com/questions/46444656/bleu-scores-could-i-use-nltk-translate-bleu-score-sentence-bleu-for-calculating

def tokenize_annotation(anns):
  text = []
  pos_text = []
  for ann in anns:
    #text.append(ann['caption'].split())
    sentence = nltk.word_tokenize(ann['caption'])
    text.append(sentence)
    pos = nltk.pos_tag(sentence)
    pos_text.append([el[1] for el in pos])
  return text,pos_text

def split_sentence(corpus = ""):
  text = []
  pos_text = []
  corpus = corpus.replace(".",". ")
  d = "."
  sentences = corpus.split(d)
  for sentence in sentences:
    #s = sentence.split()
    sentence += "."
    s = nltk.word_tokenize(sentence)
    if len(s) > 1:
      text.append(s)
      pos = nltk.pos_tag(s) #estraggo solo la pos
      pos_text.append([el[1] for el in pos])
    #return text,corpus.replace("."," ").split()
  return text,nltk.word_tokenize(corpus),pos_text

def calculate_bleu(original_text = "", generated_text = "", sentence_generated_text = "",mode = [True,True,True]):
  list_ref = []
  if mode[0]:
    #BLEU score calcolato tra il testo generato messo in una sola lista e il testo di riferimento. Il potenziale problema è che l'analisi 3-gram e 4-gram considera le frasi adiacenti come una cosa unica, quindi cerca di connettere la fine della frase 1 con l'inizio della frase 2, causando un abbassamento dello score.
    score = corpus_bleu([original_text], [generated_text],smoothing_function=SmoothingFunction().method4,emulate_multibleu=True,weights = (0.25,0.25,0.25,0.25))
    print("Overall BLEU Score = " + str(score)) 
  total_score = 0
  #BLEU score calcolato singolarmente tra una frase del testo generato e il testo di riferimento. Il problema è che tratto le frasi come parti singole, quindi l'analisi 3-gram e 4-gram non viene eseguita tra frasi diverse. L'analisi BLEU non è cumulativa
  for sentence in sentence_generated_text:
    if mode[1]:
      #print(sentence)
      score = corpus_bleu([original_text], [sentence],smoothing_function=SmoothingFunction().method4,emulate_multibleu=True,weights = (0.25,0.25,0.25,0.25))
      total_score += score
    list_ref.append(original_text)
  if mode[1]:
    print("Average on sentences BLEU Score = " + str(total_score/len(list_ref)))
  if mode[2]:
    #BLEU Score calcolato cumulativamente tra una singola frase generata e il testo di riferimento. Questa è un'analisi cumulativa che differisce dalla media dei BLEU score calcolati sulle singole frasi. Penso sia questo il modo corretto di calcolarlo
    score = corpus_bleu(list_ref, sentence_generated_text,smoothing_function=SmoothingFunction().method4,emulate_multibleu=True,weights = (0.25,0.25,0.25,0.25))
    print("Cumulative on sentences BLEU Score = " + str(score))

def BLEU_eval(category = ['outdoor'], corpus = "", ann = []): 
  original_text,pos_original = tokenize_annotation(ann)
  sentence_generated_text,generated_text,pos_sentence_generated = split_sentence(corpus = corpus)
  pos_generated = [el[1] for el in nltk.pos_tag(generated_text)]

  print("===== BLEU SCORE =====")
  calculate_bleu(original_text = original_text, generated_text = generated_text, sentence_generated_text = sentence_generated_text,mode = [True,True,True])
  print("===== SELF BLEU SCORE =====")
  calculate_bleu(original_text = generated_text, generated_text = generated_text, sentence_generated_text = sentence_generated_text, mode = [True,True,True])
  print("===== POS BLEU SCORE =====")
  calculate_bleu(original_text = pos_original, generated_text = pos_generated, sentence_generated_text = pos_sentence_generated,mode = [True,True,True])


sia in bleu che in self bleu get_scoore() e get_bleu() sembrano le funzioni principali da richiamare. Le altre sono chiamate tutte da esse. Capire la differenza tra le 2

BLEU -> applicare il sentence_bleu (o corpus_bleu) tra il testo generato e quello di partenza, bigger is better
              link texygen BLEU -> https://github.com/geek-ai/Texygen/blob/master/utils/metrics/Bleu.py
SELF BLEU -> applicare il sentence_bleu (o corpus_bleu) tra il testo generato e se stesso, lower is better (?) da controllare
              link texygen self bleu -> https://github.com/geek-ai/Texygen/blob/master/utils/metrics/SelfBleu.py
              
POS BLEU -> 
s = t1, t2, ...., tn
t1 -> significato grammaticale == Noun
t2 == Verb 
....
POS BLEU : applicare il bleu non sui token ma sulle POS (noun, verb, ...) --> ANALISI LOGICA

how to extract parts of speech (POS) --> https://medium.com/@gianpaul.r/tokenization-and-parts-of-speech-pos-tagging-in-pythons-nltk-library-2d30f70af13b

### Execution

In [ ]:
from torch.nn import functional as F
import torch

choosen_cat = ""
pretrained = False
save_model = True
if os.path.isfile('/content/drive/MyDrive/Machine Learning/Progetto/Models/vanilla10/pytorch_model.bin'):
  pretrained = True
if True:#not pretrained:
  coco,coco_ann = init_COCO()
  cats = get_categories(coco = coco)
else: #se è pretrainato uso la lista per non obbligare l'utente ad avere le annotazioni scaricate
  supercats = ['outdoor', 'person', 'kitchen', 'furniture', 'food', 'animal', 'appliance', 'accessory', 'vehicle', 'indoor', 'sports', 'electronic']
model,tokenizer,device = init(pretrained)
for cat in ['outdoor']:
  ann = get_annotation(coco = coco, coco_ann = coco_ann, catNms = [cat]) 
  if True:#not pretrained:
    finetune_v3(model = model,tokenizer = tokenizer,device = device, annotations = ann, category = cat)
if save_model:
  model.save_pretrained("/content/drive/MyDrive/Machine Learning/Progetto/Models/outdoor/person")
while choosen_cat != "exit":
  choosen_cat = choose_cat(cats)
  if choosen_cat != "exit":
    output = generate_v2(model = model, tokenizer = tokenizer, device = device, control_code = choosen_cat)
    ann = get_annotation(coco = coco, coco_ann = coco_ann, catNms = choosen_cat)
    BLEU_eval(category = choosen_cat, corpus = output, ann = ann)

In [ ]:
While True:
  a = 0

# Texygen evaluation

**Where the execution being stopped raising errors concerned about tensorflow,  replace `import tensorflow as tf` with `import tensorflow.compat.v1 as tf` and adding `tf.disable_v2_behavior()` before the code**





In [ ]:
def download_texygen():
  !pip install colorama

  import nltk
  nltk.download('punkt')
  project_path = '/content/drive/MyDrive/Machine Learning/Progetto'
  os.chdir(project_path)
  if not os.path.isdir('/content/drive/MyDrive/Machine Learning/Progetto/Texygen'):
    !git clone https://github.com/geek-ai/Texygen.git

def run_texygen(gan = "mle", training = "oracle", data = "./data/image_coco.txt"):
  texygen_path = '/content/drive/MyDrive/Machine Learning/Progetto/Texygen'
  os.chdir(texygen_path)
  !python3 main.py -g $gan -t $training -d $data

def plot_results_texygen(gan = "mle", training = "oracle", data = "./data/image_coco.txt"): 
  import pandas as pd
  import matplotlib.pyplot as plt

  base_path = "/content/drive/MyDrive/Machine Learning/Progetto/Texygen/experiment-log-" + gan
  suffix_path = "-" + training + ".csv" if training != "oracle" else ".csv"
  result_file_path = base_path + suffix_path

  print("READING FILE: " + result_file_path)
  print("")
  results = pd.read_csv(result_file_path)
  results.dropna(axis = "columns", inplace = True)
  print(results)

  epochs = results.iloc[:, 0]
  metrics = {}
  for index in range(1, len(results.columns)):  
    metrics["metric{0}".format(index)] = results.iloc[:, index]

  with plt.style.context("Solarize_Light2"):
    for key, value in metrics.items():
      plt.plot(epochs, value, "-b", marker="o", label = value.name)
      plt.legend()

  plt.show()

In [ ]:
download_texygen()
run_texygen("leakgan", "real")

In [ ]:
plot_results_texygen("seqgan", "real")

#API e salesforce

### Prepare model

In [ ]:
from shutil import copyfile
import gc

# --- INITIALIZING SOME CONSTANT ----

lowerMemory = False
mixed = False
project_path = "/content/drive/MyDrive/Machine Learning/Progetto/"
ctrl_path = project_path + "ctrl/"
training_path = ctrl_path + "training_utils/"
pretrained_model_dir = "model/"
model_name = "model.ckpt-684000.data-00000-of-00001"
model_dir = "seqlen256_36layers_v0.ckpt/"  

def Patch_Keras(old_path = ctrl_path):
  os.chdir(ctrl_path)
  !patch -b /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/keras.py estimator.patch
  os.chdir(old_path)

def init_no_API():
  os.chdir(project_path) #%cd project_path
  !nvidia-smi
  !pip2 install tensorflow-gpu==1.14 #installo le dipendenze
  prova = training_path + "prova1.txt"
  !rm -rf ctrl
  !rm -rf prova
  !git clone https://github.com/salesforce/ctrl #clono CTRL
  os.chdir(ctrl_path) #%cd ctrl_path
  if lowerMemory == True:
    Path_Keras(ctrl_path)
    !git checkout lower_memory
  else:
    !pip install torch
    !git checkout master
  !pip2 install fastBPE

  if mixed == False:
    if os.path.isfile(project_path + model_dir + model_name) == False:
      os.chdir(project_path)#%cd project_path
      !pip2 install gsutil #gsutil is a Python application that lets you access Cloud Storage from the command line. You can use gsutil to do a wide range of bucket and object management tasks
      !gsutil -m cp -r gs://sf-ctrl/seqlen256_36layers_v0.ckpt/ .
      #!gsutil -m cp -r gs://sf-ctrl/seqlen256_v1.ckpt . #48 layer
      #()
  else:
    print("MIXED @@@@@@@@@@@@@@@@")
    #init_mixed()

def generate_no_API():
  gc.collect()
  os.chdir(ctrl_path)#%cd ctrl_path
  if lowerMemory == True:
    !python2 generation.py --model ../seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001
  else:
    !python2 pytorch_generation.py --temperature 0.5 --model ../seqlen256_v1.ckpt/model.ckpt-684000.data-00000-of-00001

def init_API():
  !pip install transformers
  from transformers import TFCTRLLMHeadModel, CTRLConfig, CTRLTokenizer
  import torch
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  tokenizer = CTRLTokenizer.from_pretrained('ctrl')
  model = TFCTRLLMHeadModel.from_pretrained('ctrl', config = CTRLConfig(n_layer = 10))
  
  return model,tokenizer,device

def generate_API(model,tokenizer,device):
  inputs = tokenizer("Politics Title:", return_tensors="tf")
  outputs = model(inputs)
  return outputs

def finetune_CTRL_API():
  #farò il finetune e salverò il modello su drive, cancellando la cartella del API = false
  #nel metodo API = False dovrò cancellare questo modello
  i = 50

def Create_tf_records(file_list = [("outdoor","prova_1.txt")]): 
  #avrò una lista con [(nome_label,path_file),...]. dovrò leggere path_file da google drive e produrrò una lista con il path al file contenente il tf record
  Patch_Keras(training_path)  
  #os.system("wget -O moby_dick.txt https://www.gutenberg.org/files/2701/2701-0.txt") #DEBUG, per le prove finchè non avrò la lista
  #mi salva i record in moby_dick.txt.tfrecords
  for (control_f,path_f) in file_list:
    copyfile(project_path + path_f, training_path + path_f)
    path_file = training_path + path_f + ".tfrecords"
    if os.path.isfile(path_file) == False:
      #se non esiste il file tfrecords devo crearlo
      command_create_tf = "python2 make_tf_records.py --text_file {0} --control_code {1} --sequence_len 256".format(path_f,control_f)
      subprocess.check_output(command_create_tf,shell=True)
      print("Creato file con control code {0}".format(control_f))

def finetune_CTRL_no_API(file_list = []):
  Create_tf_records()
  #train_cmd = "python2 training.py --model_dir {} --iterations 1".format(project_path.replace(" ","\\ ") + model_dir)
  print("START_TRAINING")
  gc.collect()
  #subprocess.check_output(train_cmd,shell = True)
  !python2 training.py --model_dir /content/drive/MyDrive/Machine\ Learning/Progetto/seqlen256_36layers_v0.ckpt/ --iterations 1

### Text generation

In [ ]:
use_api = True
finetune_CTRL = True
generate_CTRL = False
if use_api == False:
  init_no_API()
  if finetune_CTRL == True:
    finetune_CTRL_no_API()
  if generate_CTRL == True:
    generate_no_API()
else:
  model,tokenizer,device = init_API()
  output = generate_API(model = model,tokenizer = tokenizer, device = device)